In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")

from src.model import UNet
from src.dataset import get_load_data
from torch.utils.data import DataLoader
from src.predict import predict
import torch
import numpy as np
import torch.nn.functional as F

In [2]:
unet_model = UNet(num_classes=21)
train, test = get_load_data(root = "../../data", dataset = "VOCSegmentation", download = False)  
test_set = DataLoader(test, batch_size=1)

trained_model_path = "../model_weights/model_weights.pt"
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = torch.load(trained_model_path, map_location=torch.device(device))
model.eval()
imgs, smnts = next(iter(test_set))
imgs, smnts = imgs.to(device), smnts.to(device)
preds = model(imgs)

In [4]:
smnts = (smnts * 255).type(torch.int64)
preds = preds.argmax(dim=1).type(torch.int64)


In [5]:
preds = F.one_hot(preds, 21).float()
smnts = F.one_hot(smnts, 21).float()

In [6]:
preds = preds.permute(0, 3, 1, 2)
smnts = smnts.squeeze(0).permute(0, 3, 1, 2)

In [7]:
preds = preds.squeeze(0)
smnts = smnts.squeeze(0)